In [18]:
import numpy as np
import optuna
from optuna import Trial

from evo import evo_alg
from shpm2 import SHPM

import random

from copy import deepcopy
import matplotlib.pyplot as plt

In [19]:
from optuna.visualization import plot_parallel_coordinate, plot_param_importances, plot_contour, plot_slice

In [20]:
random.seed(42)

In [21]:
# consistent parameters
n = 4

population_size = 100
parents = 5
max_iters = 50

In [22]:
shpm = SHPM(n)

In [23]:
# targets:
n_targets = 10

P_targets = [
    shpm.random_point() for i in range(n_targets)
]

In [24]:
A_targets = [
    shpm.assemble(P) for P in P_targets
]

In [25]:
initial_population = [
    shpm.random_point() for i in range(population_size)
]

In [26]:
def trial_objective(trial: Trial):

    # suggest parameter values            
    params = {
        ## mutation magnitudes
        "Q_mut_mag": trial.suggest_float("Q_mut_mag", 1e-4, 1e-1, log=True),
        "UT_mut_mag": trial.suggest_float("UT_mut_mag", 1e-4, 1e-1, log=True),
        "SD_mut_mag": trial.suggest_float("SD_mut_mag", 1e-4, 1e-1, log=True),
        "HD_mut_mag": trial.suggest_float("HD_mut_mag", 1e-4, 1e-1, log=True),
        ## event probabilities
        "Q_swap_chance": trial.suggest_float("Q_swap_chance", 1e-4, 1e-1, log=False),
        "crazy_Q_xv_chance": trial.suggest_float("crazy_Q_xv_chance", 1e-2, 2e-1, log=False),
        "HD_1x1_merge_chance": trial.suggest_float("HD_1x1_merge_chance", 1e-2, 5e-1, log=False),
        "HD_2x2_split_chance": trial.suggest_float("HD_2x2_split_chance", 1e-2, 5e-1, log=False),
        "HD_eig_swap_chance": trial.suggest_float("HD_eig_swap_chance", 1e-2, 5e-1, log=False)    
    }
    crossover_chance = trial.suggest_float("crossover_chance", 1e-1, 6e-1, log=True)

    shpm_trial = SHPM(n, **params)

    best_objectives = []

    for target_idx, A_target in enumerate(A_targets):

        A_target_norm = np.linalg.norm(A_target)

        def evo_objective(P):            
            
            A = shpm.assemble(P)
            distance = np.linalg.norm(A - A_target) / A_target_norm
            return distance

        _, _, _, population_objectives = evo_alg(
            deepcopy(initial_population),
            parents, population_size,
            evo_objective,
            shpm.mutate, shpm.crossover,
            s=1.0, c=crossover_chance,
            maxiter=max_iters, eps=1e-5
        )

        best_objectives.append(population_objectives[-1][0])
    
    best_obj_avg = np.sum(best_objectives) / n_targets

    return best_obj_avg

In [27]:
study = optuna.create_study()

[I 2025-05-16 13:12:18,989] A new study created in memory with name: no-name-bf0291dc-cc2e-47c6-8846-d318ced5420b


In [28]:
study.optimize(
    trial_objective,
    48,
    n_jobs=8,
    show_progress_bar=True    
)

Best trial: 1. Best value: 0.212875:   2%|▏         | 1/48 [00:49<39:07, 49.95s/it]

[I 2025-05-16 13:13:08,939] Trial 1 finished with value: 0.21287461200018734 and parameters: {'Q_mut_mag': 0.00046352036047095504, 'UT_mut_mag': 0.00015932099841390186, 'SD_mut_mag': 0.009427780275002429, 'HD_mut_mag': 0.0016532705279771274, 'Q_swap_chance': 0.09273772595060485, 'crazy_Q_xv_chance': 0.04646152046882831, 'HD_1x1_merge_chance': 0.17339826095289576, 'HD_2x2_split_chance': 0.191096470395179, 'HD_eig_swap_chance': 0.44771615231072875, 'crossover_chance': 0.5661314823355694}. Best is trial 1 with value: 0.21287461200018734.


Best trial: 1. Best value: 0.212875:   4%|▍         | 2/48 [00:59<20:04, 26.18s/it]

[I 2025-05-16 13:13:18,483] Trial 5 finished with value: 0.21419898167503942 and parameters: {'Q_mut_mag': 0.051171671912101854, 'UT_mut_mag': 0.0041279051488892025, 'SD_mut_mag': 0.0005599001515488996, 'HD_mut_mag': 0.012179538849341929, 'Q_swap_chance': 0.012263329711685868, 'crazy_Q_xv_chance': 0.08874996901799534, 'HD_1x1_merge_chance': 0.49258522544553646, 'HD_2x2_split_chance': 0.11727133090868941, 'HD_eig_swap_chance': 0.39442287560334005, 'crossover_chance': 0.47465069461998494}. Best is trial 1 with value: 0.21287461200018734.


Best trial: 1. Best value: 0.212875:   6%|▋         | 3/48 [01:00<11:01, 14.71s/it]

...

[I 2025-05-16 13:20:52,257] Trial 45 finished with value: 0.18163967395415637 and parameters: {'Q_mut_mag': 0.0003937978873343216, 'UT_mut_mag': 0.0021650080177622304, 'SD_mut_mag': 0.00023901370305893328, 'HD_mut_mag': 0.0002596500363972267, 'Q_swap_chance': 0.08892658793200114, 'crazy_Q_xv_chance': 0.09795676385753528, 'HD_1x1_merge_chance': 0.32530560782524554, 'HD_2x2_split_chance': 0.15310758665851432, 'HD_eig_swap_chance': 0.332830788477794, 'crossover_chance': 0.19051636200243743}. Best is trial 6 with value: 0.16755104642772442.


Best trial: 6. Best value: 0.167551:  98%|█████████▊| 47/48 [08:34<00:04,  4.75s/it]

[I 2025-05-16 13:20:53,172] Trial 46 finished with value: 0.18899705926051982 and parameters: {'Q_mut_mag': 0.00034386041994920126, 'UT_mut_mag': 0.0005674804112067826, 'SD_mut_mag': 0.0002561711914618589, 'HD_mut_mag': 0.023364408960226686, 'Q_swap_chance': 0.09000490692394324, 'crazy_Q_xv_chance': 0.10024728522131704, 'HD_1x1_merge_chance': 0.32612652954194327, 'HD_2x2_split_chance': 0.42114568920971684, 'HD_eig_swap_chance': 0.3529993207167341, 'crossover_chance': 0.19683396921041835}. Best is trial 6 with value: 0.16755104642772442.


Best trial: 6. Best value: 0.167551: 100%|██████████| 48/48 [08:34<00:00, 10.72s/it]

[I 2025-05-16 13:20:53,743] Trial 47 finished with value: 0.19074487269662102 and parameters: {'Q_mut_mag': 0.00029695607786870586, 'UT_mut_mag': 0.00019206632663945264, 'SD_mut_mag': 0.0002472202958070953, 'HD_mut_mag': 0.017879530929344602, 'Q_swap_chance': 0.08955325344476148, 'crazy_Q_xv_chance': 0.09966997314397852, 'HD_1x1_merge_chance': 0.2915339219300671, 'HD_2x2_split_chance': 0.2714784099850293, 'HD_eig_swap_chance': 0.41904381499305443, 'crossover_chance': 0.18862144388350824}. Best is trial 6 with value: 0.16755104642772442.


In [29]:
study.best_params

{'Q_mut_mag': 0.031119207418375497,
 'UT_mut_mag': 0.015375819408866256,
 'SD_mut_mag': 0.0008035457619080877,
 'HD_mut_mag': 0.00027540338292686976,
 'Q_swap_chance': 0.09632035425273437,
 'crazy_Q_xv_chance': 0.09009889291433247,
 'HD_1x1_merge_chance': 0.44506374930225395,
 'HD_2x2_split_chance': 0.24595115022034447,
 'HD_eig_swap_chance': 0.01382207302286094,
 'crossover_chance': 0.15969873073468194}

In [30]:
# ax = plot_parallel_coordinate(study)
# ax.figure.set_size_inches((15, 5))
# plt.show()
plot_parallel_coordinate(study)

In [31]:
plot_contour(study, params=["crossover_chance", "crazy_Q_xv_chance", "Q_swap_chance"])

In [32]:
plot_param_importances(study)

In [33]:
plot_slice(study)